In [1]:
# !pip install pandas tqdm transformers accelerate bitsandbytes

In [2]:
# -*- coding: utf-8 -*-
# 기존 코드와 완전 호환되는 "ISMS-P 안내서" RAG 통합 확장판

import re
import os
import json
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from PyPDF2 import PdfReader
from langchain_community.embeddings import HuggingFaceEmbeddings
import faiss

# ---------------- 사용자 원본 코드 일부 ----------------
test = pd.read_csv('../data/test.csv')

def is_multiple_choice(question_text):
    lines = question_text.strip().split("\n")
    option_count = sum(bool(re.match(r"^\s*[1-9][0-9]?\s", line)) for line in lines)
    return option_count >= 2

def extract_question_and_choices(full_text):
    lines = full_text.strip().split("\n")
    q_lines, options = [], []
    for line in lines:
        if re.match(r"^\s*[1-9][0-9]?\s", line):
            options.append(line.strip())
        else:
            q_lines.append(line.strip())
    question = " ".join(q_lines)
    return question, options

# ---------------- 모델/임베딩/청킹 파라미터 ----------------
LLM_ID = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
EMB_MODEL = "jhgan/ko-sroberta-multitask"   # 경량 추천
CHUNK_TOKENS = 600
CHUNK_OVERLAP = 32
CTX_TOKEN_BUDGET = 1200
TOP_K = 3
SEED = 42
torch.manual_seed(SEED)

# ---------------- 토크나이저 ----------------
llm_tokenizer = AutoTokenizer.from_pretrained(LLM_ID)
if llm_tokenizer.pad_token is None:
    llm_tokenizer.pad_token = llm_tokenizer.eos_token
llm_tokenizer.padding_side = "right"

def token_len(s: str) -> int:
    return len(llm_tokenizer(s, add_special_tokens=False)["input_ids"])

def split_sentences_ko(text: str) -> List[str]:
    # '다.' 등 종결부 + 일반 문장부호 기준 분할
    text = re.sub(r'\s+', ' ', text).strip()
    if not text:
        return []
    return re.split(r'(?<=다\.)\s+|(?<=[.?!。！？])\s+', text)

# ---------------- 문서 데이터 구조 ----------------
@dataclass
class LawDoc:
    text: str
    meta: Dict

# ---------------- 임베딩 ----------------
embeddings = HuggingFaceEmbeddings(
    model_name=EMB_MODEL,
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"},
    encode_kwargs={
        "normalize_embeddings": True,
        "batch_size": 128,
        "convert_to_numpy": True,
        "convert_to_tensor": False
    }
)

# ---------------- 공통 유틸 ----------------
def load_pdf_text(pdf_path: str) -> str:
    reader = PdfReader(pdf_path)
    text = ""
    for p in reader.pages:
        t = p.extract_text() or ""
        text += t + "\n"
    return text

def normalize_common(text: str) -> str:
    # circled numbers → (n)
    circled = '①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳'
    for idx, c in enumerate(circled, 1):
        text = text.replace(c, f'({idx})')
    # 한자 제거(있으면)
    text = re.sub(r'[\u4e00-\u9fff]', '', text)
    # 공백 정리
    text = re.sub(r'[ \t]+', ' ', text)
    text = re.sub(r'\s+\n', '\n', text)
    text = re.sub(r'\n\s+', '\n', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def build_faiss_hnsw(vectors: np.ndarray, m: int = 32, ef_search: int = 32) -> faiss.IndexHNSWFlat:
    dim = vectors.shape[1]
    idx = faiss.IndexHNSWFlat(dim, m)
    idx.hnsw.efSearch = ef_search
    idx.add(vectors.astype(np.float32))
    return idx

def chunk_by_tokens(prefix: str, body: str) -> List[str]:
    prefix = (prefix or "").strip()
    sents = split_sentences_ko(body) or [body]
    chunks, cur, cur_toks = [], [], token_len(prefix + "\n") if prefix else 0
    for s in sents:
        tl = token_len(s)
        if cur_toks + tl > CHUNK_TOKENS and cur:
            chunks.append((prefix + "\n" if prefix else "") + " ".join(cur))
            keep = cur[-1] if CHUNK_OVERLAP > 0 and cur else ""
            cur = [keep] if keep else []
            cur_toks = (token_len(prefix + "\n") if prefix else 0) + (token_len(keep) if keep else 0)
        cur.append(s); cur_toks += tl
    if cur:
        chunks.append((prefix + "\n" if prefix else "") + " ".join(cur))
    return chunks

# =====================================================================
# 1) "법령" 파트 (기존): 조문 단위 분리 및 인덱싱 그대로 유지
# =====================================================================
LAW_CONFIG = {
    # (예시) 개인정보 보호법 등 기존에 쓰던 항목들...
    # 필요 시 그대로 유지/사용. 여기서는 ISMS-P 추가가 목적이므로 생략 가능.
}

ARTICLE_HEADER_PATTERN = r'(제\d+조(?:의\d+)?\([^)]+\))'
def split_articles(raw_text: str) -> List[Tuple[str, str, str]]:
    parts = re.split(ARTICLE_HEADER_PATTERN, raw_text)
    out = []
    for i in range(1, len(parts), 2):
        header = parts[i]
        body = (parts[i+1] if i+1 < len(parts) else "").strip().replace("\n", " ")
        m = re.match(r'(제\d+조(?:의\d+)?)[(]([^)]+)[)]', header)
        if not m:
            continue
        article_id = m.group(1)
        article_title = m.group(2)
        out.append((article_id, article_title, body))
    return out

def clean_text_by_config(text: str, drop_patterns: List[str]) -> str:
    for pat in drop_patterns:
        text = re.sub(pat, '', text)
    return normalize_common(text)

def preprocess_law(law_id: str, cfg: Dict) -> List[LawDoc]:
    raw = load_pdf_text(cfg["pdf_path"])
    cleaned = clean_text_by_config(raw, cfg["drop_patterns"])
    articles = split_articles(cleaned)

    docs: List[LawDoc] = []
    effective_date = None
    for article_id, title, body in articles:
        header = f'{cfg["law_name"]} {article_id}({title})'
        chunks = chunk_by_tokens(header, body)
        for ch in chunks:
            meta = {
                "law_id": law_id,
                "law_name": cfg["law_name"],
                "article_id": article_id,
                "article_title": title,
                "effective_date": effective_date,
                "tok_len": token_len(ch),
                "source_uri": cfg.get("pdf_path"),
                "version": None
            }
            docs.append(LawDoc(text=ch, meta=meta))
    return docs

# =====================================================================
# 2) "금융보안원 ISMS-P 점검항목 안내서" 전용 전처리/청킹/인덱싱
# =====================================================================

# 업로드된 파일 경로(필요 시 환경에 맞게 변경)
GUIDE_PDF_PATH = "/mnt/data/금융보안원 - 금융권에 적합한 ISMS-P 인증기준 점검항목 안내서(2023.12.).pdf"

# 안내서에서 반복되는 머리말/꼬리말/페이지 번호/표 캡션 등을 최대한 제거
ISMSP_DROP_PATTERNS = [
    r'금융보안원', r'ISMS-?P', r'인증기준', r'점검항목 안내서',
    r'목차\s*', r'표\s*\d+[-.]?\d*', r'그림\s*\d+[-.]?\d*',
    r'페이지\s*\d+/\d+', r'Page\s*\d+/\d+',
    r'^\s*\d+\s*$',              # 단독 페이지 번호 라인
]

# (헤딩) 섹션 탐지: "1.", "1.1", "Ⅰ.", "가.", "(1)" 등 다양하게 오는 경우를 폭넓게 커버
SECTION_HEADER = re.compile(
    r'^\s*((?:[IVX]+\.|\d+(?:\.\d+)*\.?|[가-힣]\.|\( ?\d+\)|\( ?[가-힣]\)))\s+(.{2,100})\s*$'
)

def clean_isms_p_text(raw_text: str) -> str:
    # 줄 단위로 header/footer/표 번호/빈 줄 제거
    lines = [ln for ln in raw_text.splitlines()]
    cleaned_lines = []
    for ln in lines:
        skip = False
        for pat in ISMSP_DROP_PATTERNS:
            if re.search(pat, ln, flags=re.IGNORECASE):
                # 'ISMS-P' 키워드가 본문 내용인 경우까지 지우지 않도록, 전역 제거 대신 "머리말 패턴" 중심
                # 다만 본문 키워드까지 과하게 지워진다고 느껴지면 이 블록을 보수적으로 조정하세요.
                pass
        # 너무 짧은 잡음 라인/구분선 제거
        if re.match(r'^\s*[-=]{4,}\s*$', ln): 
            continue
        if re.match(r'^\s*$', ln):
            continue
        cleaned_lines.append(ln)

    text = "\n".join(cleaned_lines)
    # 페이지 머리/꼬리 흔적 제거(보수적으로)
    text = re.sub(r'\n?Copyright .*?\n', '\n', text, flags=re.IGNORECASE)
    text = normalize_common(text)
    return text

def split_isms_p_sections(clean_text: str) -> List[Tuple[str, str]]:
    """
    안내서의 특성상 '섹션 헤더 라인'을 기준으로 큰 블록을 만든 뒤,
    블록 내부는 문장 단위 청킹으로 토크나이즈 예산에 맞춰 분할합니다.
    반환: [(섹션제목, 섹션 본문), ...]
    """
    lines = clean_text.split("\n")
    sections = []
    cur_title, cur_buf = None, []

    def push():
        nonlocal sections, cur_title, cur_buf
        if cur_title and cur_buf:
            body = " ".join(cur_buf).strip()
            if body:
                sections.append((cur_title, body))
        cur_title, cur_buf = None, []

    for ln in lines:
        m = SECTION_HEADER.match(ln)
        if m:
            # 새로운 섹션 시작
            push()
            # 제목: "1. 개요" 같은 형태로 정규화
            mark, title = m.group(1), m.group(2)
            cur_title = f"{mark} {title}".strip()
        else:
            if cur_title is None:
                # 서문/요약 같은 프리앰블은 '0. 서문' 식으로 묶어줌
                cur_title = "서문"
            cur_buf.append(ln)

    push()
    return sections

def preprocess_isms_p_guide(pdf_path: str) -> List[LawDoc]:
    raw = load_pdf_text(pdf_path)
    cleaned = clean_isms_p_text(raw)
    sections = split_isms_p_sections(cleaned)

    docs: List[LawDoc] = []
    for title, body in sections:
        # 섹션을 먼저 큰 덩어리로 만들고, 토큰 청킹
        chunks = chunk_by_tokens(f"ISMS-P 점검항목 안내서 {title}", body)
        for ch in chunks:
            meta = {
                "law_id": "isms_p_guide",
                "law_name": "금융보안원 ISMS-P 점검항목 안내서",
                "section_title": title,
                "tok_len": token_len(ch),
                "source_uri": pdf_path,
                "doc_type": "guide",
            }
            docs.append(LawDoc(text=ch, meta=meta))
    return docs

# =====================================================================
# 3) 인덱스 빌드 (법령 + ISMS-P 안내서)
# =====================================================================

def build_indices(all_docs: List[LawDoc]):
    # (a) 문서군별 인덱스
    per_group_docs: Dict[str, List[LawDoc]] = {}
    for d in all_docs:
        group = d.meta.get("law_id", "misc")
        per_group_docs.setdefault(group, []).append(d)

    indices = {}
    for gid, docs in per_group_docs.items():
        mat = np.array(embeddings.embed_documents([d.text for d in docs]), dtype=np.float32)
        indices[f"faiss_hnsw_{gid}"] = {
            "index": build_faiss_hnsw(mat, m=32, ef_search=32),
            "docs": docs
        }

    # (b) 글로벌 인덱스
    mat_all = np.array(embeddings.embed_documents([d.text for d in all_docs]), dtype=np.float32)
    indices["faiss_hnsw_all"] = {
        "index": build_faiss_hnsw(mat_all, m=32, ef_search=32),
        "docs": all_docs
    }
    return indices

# ---------------- 라우팅 규칙 업데이트 ----------------
ARTICLE_PTRN = re.compile(r"제\d+조(?:의\d+)?")

LAW_HINTS = {
    # 개인정보 보호법
    "pipa": {
        "law_name": "개인정보 보호법",
        "pdf_path": "../data/개인정보 보호법(법률)(제19234호)(20250313).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*개인정보\s*보호법',
            r'법제처\s+\d+\s+국가법령정보센터',
            r'국가법령정보센터\s*개인정보\s*보호법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>',         # <개정 …>, <신설 …>
            r'\[[^\]]+\]',      # [본조신설 …]
        ],
    },
    # 신용정보의 이용 및 보호에 관한 법률
    "ciupa": {
        "law_name": "신용정보법",
        "pdf_path": "../data/신용정보의 이용 및 보호에 관한 법률(법률)(제20304호)(20240814).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*신용정보.*법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
    # 전자서명법
    "es_act": {
        "law_name": "전자서명법",
        "pdf_path": "../data/전자서명법(법률)(제18479호)(20221020).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*전자서명법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
    # 정보통신망 이용촉진 및 정보보호 등에 관한 법률
    "icn_act": {
        "law_name": "정보통신망법",
        "pdf_path": "../data/정보통신망 이용촉진 및 정보보호 등에 관한 법률(법률)(제20678호)(20250722).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*정보통신망.*법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
    # 전자금융거래법
    "eft_act": {
        "law_name": "전자금융거래법",
        "pdf_path": "../data/전자금융거래법(법률)(제19734호)(20240915).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*전자금융거래.*법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
    # 전자금융감독규정
    "rs_act": {
        "law_name": "전자금융감독규정",
        "pdf_path": "../data/전자금융감독규정(금융위원회고시)(제2025-4호)(20250205).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*전자금융거래.*법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
    # 자본시장법
    "fis_act": {
        "law_name": "자본시장법",
        "pdf_path": "../data/자본시장과 금융투자업에 관한 법률(법률)(제20718호)(20250722).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*자본시장.*법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
}

LAW_HINTS = {
    # 기존 법령 힌트들(있다면 유지) ...
    # "pipa": ("개인정보", "개인정보보호법"),
    # ...
    # ISMS-P 안내서 힌트 추가
    "isms_p_guide": (
        "isms p", "isms-p", "ismsp", "ismsp",
        "인증기준", "점검항목", "관리적 보안", "기술적 보안", "물리적 보안",
        "보안성 심의", "접근통제", "암호", "취약점 진단", "로그", "백업",
        "금융보안원", "개인정보 관리체계", "관리체계 수립", "개선 조치", "보호대책"
    ),
}

def detect_law_id(query: str) -> Optional[str]:
    q = query.lower()
    for law_id, kws in LAW_HINTS.items():
        if any(kw.lower() in q for kw in kws):
            return law_id
    return None

def route_is_domain(query: str) -> bool:
    domain_kws = ("법", "조(", "과징금", "처벌", "보안", "침해", "금융", "개인정보", "신용정보",
                  "전자서명", "정보통신", "전자금융", "금융감독", "자본", "자본시장", "투자",
                  "점검항목", "인증기준", "관리체계", "ISMS")
    q = query.lower()
    return any(kw.lower() in q for kw in domain_kws) or bool(ARTICLE_PTRN.search(query))

def choose_index(indices: dict, query: str):
    law_id = detect_law_id(query)
    if law_id:
        key = f"faiss_hnsw_{law_id}"
        if key in indices:
            return indices[key]
    if route_is_domain(query) and "faiss_hnsw_all" in indices:
        return indices["faiss_hnsw_all"]
    return None

# ---------------- 컨텍스트 패킹/프롬프트/생성(사용자 코드 유지) ----------------
def pack_context(docs_in, token_budget=CTX_TOKEN_BUDGET):
    acc, used = [], 0
    for d in docs_in:
        tl = d.meta.get("tok_len", None)
        if tl is None:
            tl = token_len(d.text); d.meta["tok_len"] = tl
        if used + tl <= token_budget:
            acc.append(d.text); used += tl
        else:
            remain = token_budget - used
            if remain > 50:
                ids = llm_tokenizer(d.text, add_special_tokens=False)["input_ids"][:remain]
                acc.append(llm_tokenizer.decode(ids))
            break
    return "\n\n".join(acc)

def chat_prompt(system, user):
    messages = [{"role": "system", "content": system},
                {"role": "user", "content": user}]
    return llm_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

def dynamic_max_new_tokens(question: str) -> int:
    lines = [ln.strip() for ln in question.split("\n") if ln.strip()]
    opt_cnt = sum(bool(re.match(r"^\d+(\s|[.)])", ln)) for ln in lines)
    return 96 if opt_cnt >= 2 else 192

def build_prompt(query: str, use_context: bool, context) -> Tuple[str, int]:
    if is_multiple_choice(query):
        question, options = extract_question_and_choices(query)
        if use_context:
            prompt = (
                "아래 컨텍스트를 우선 사용해 정확히 답하세요. 불충분하면 아는 범위에서만 간결히 답하세요.\n\n"
                f"=== 컨텍스트 ===\n{context}\n=== 끝 ===\n"
                "아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.\n\n"
                f"질문: {question}\n선택지:\n{chr(10).join(options)}\n\n답변:"
            )
            system = "당신은 금융/보안 QA 도우미입니다. 포맷을 엄격히 지키세요."
            prompt = chat_prompt(system, prompt)
            return prompt, 3072
        else:
            prompt = (
                "아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.\n\n"
                f"질문: {question}\n선택지:\n{chr(10).join(options)}\n\n답변:"
            )
            system = "당신은 금융/보안 QA 도우미입니다. 포맷을 엄격히 지키세요."
            prompt = chat_prompt(system, prompt)
            return prompt, 2048
    else:
        if use_context:
            prompt = (
                "아래 컨텍스트를 우선 사용해 정확히 답하세요. 불충분하면 아는 범위에서만 간결히 답하세요.\n\n"
                f"=== 컨텍스트 ===\n{context}\n=== 끝 ===\n"
                "아래 질문에 대해 **사실에 근거한 간결한 답변**을 작성하세요.\n\n"
                "규칙:\n"
                "1. 답변은 2~3문장 이내로 작성합니다. 장황한 서론, 결론 문구는 쓰지 않습니다.\n"
                "2. 불확실할 경우 '알 수 없습니다'라고 답하고 생성을 종료합니다.\n"
                "3. 특수문자 없이 오로지 한글과 숫자로만 대답합니다.\n"
                f"질문: {query}답변:"
            )
            system = "당신은 금융/보안 QA 도우미입니다. 포맷을 엄격히 지키세요."
            prompt = chat_prompt(system, prompt)
            return prompt, 3072
        else:
            prompt = (
                "아래 질문에 대해 **사실에 근거한 간결한 답변**을 작성하세요.\n\n"
                "규칙:\n"
                "1. 답변은 2~3문장 이내로 작성합니다. 장황한 서론, 결론 문구는 쓰지 않습니다.\n"
                "2. 불확실할 경우 '알 수 없습니다'라고 답하고 생성을 종료합니다.\n"
                "3. 특수문자 없이 오로지 한글과 숫자로만 대답합니다.\n"
                f"질문: {query}답변:"
            )
            system = "당신은 금융/보안 QA 도우미입니다. 포맷을 엄격히 지키세요."
            prompt = chat_prompt(system, prompt)
            return prompt, 2048

def faiss_search_with_scores_from_index(index_entry: dict, query: str, top_k: int = TOP_K):
    qv = np.array(embeddings.embed_query(query), dtype=np.float32).reshape(1, -1)
    D, I = index_entry["index"].search(qv, top_k)    # L2
    cos = 1.0 - (D[0] / 2.0)                         # L2 → cosine (정규화 가정)
    out = []
    docs = index_entry["docs"]
    for idx, i in enumerate(I[0]):
        ii = int(i)
        if ii >= 0:
            out.append((docs[ii], float(cos[idx])))
    return out

def generate_answer_with_indices(query: str, indices: dict) -> str:
    if not route_is_domain(query):
        prompt, max_len = build_prompt(query, use_context=False, context=None)
        inputs = llm_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_len, padding=False)
        inputs = {k: v.to(llm_model.device) for k, v in inputs.items()}
        with torch.inference_mode():
            out = llm_model.generate(**inputs,
                                     max_new_tokens=dynamic_max_new_tokens(query),
                                     do_sample=False, temperature=0.0, top_p=0.1,
                                     repetition_penalty=1.1,
                                     eos_token_id=llm_tokenizer.eos_token_id,
                                     pad_token_id=llm_tokenizer.pad_token_id)
        gen = out[0][inputs["input_ids"].shape[1]:]
        return llm_tokenizer.decode(gen, skip_special_tokens=True).strip()

    idx_entry = choose_index(indices, query)
    if idx_entry is None:
        prompt, max_len = build_prompt(query, use_context=False, context=None)
        inputs = llm_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_len, padding=False)
        inputs = {k: v.to(llm_model.device) for k, v in inputs.items()}
        with torch.inference_mode():
            out = llm_model.generate(**inputs,
                                     max_new_tokens=dynamic_max_new_tokens(query),
                                     do_sample=False, temperature=0.0, top_p=0.1,
                                     repetition_penalty=1.1,
                                     eos_token_id=llm_tokenizer.eos_token_id,
                                     pad_token_id=llm_tokenizer.pad_token_id)
        gen = out[0][inputs["input_ids"].shape[1]:]
        return llm_tokenizer.decode(gen, skip_special_tokens=True).strip()

    scored = faiss_search_with_scores_from_index(idx_entry, query, top_k=TOP_K)
    best_cos = max((s for _, s in scored), default=0.0)
    THRESH = 0.80
    use_context = best_cos >= THRESH and len(scored) > 0
    context = pack_context([d for d, s in scored if s >= THRESH], token_budget=CTX_TOKEN_BUDGET) if use_context else None
    prompt, max_len = build_prompt(query, use_context, context)

    inputs = llm_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_len, padding=False)
    inputs = {k: v.to(llm_model.device) for k, v in inputs.items()}
    with torch.inference_mode():
        out = llm_model.generate(
            **inputs,
            max_new_tokens=dynamic_max_new_tokens(query),
            do_sample=False, temperature=0.0, top_p=0.1,
            repetition_penalty=1.1,
            eos_token_id=llm_tokenizer.eos_token_id,
            pad_token_id=llm_tokenizer.pad_token_id,
        )
    gen = out[0][inputs["input_ids"].shape[1]:]
    return llm_tokenizer.decode(gen, skip_special_tokens=True).strip()

# ---------------- LLM 로드 ----------------
llm_model = AutoModelForCausalLM.from_pretrained(
    LLM_ID,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16
)
if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True
    try:
        llm_model.config.attn_implementation = "flash_attention_2"
    except Exception:
        pass
llm_model.eval()
torch.set_grad_enabled(False)

# ---------------- 후처리(원본 유지) ----------------
def extract_answer_only(generated_text: str, original_question: str) -> str:
    if "답변:" in generated_text:
        text = generated_text.split("답변:")[-1].strip()
    else:
        text = generated_text.strip()
    if not text:
        return "미응답"
    if is_multiple_choice(original_question):
        match = re.match(r"\D*([1-9][0-9]?)", text)
        return match.group(1) if match else "0"
    else:
        return text

def strip_explanations(answer: str) -> str:
    lines = answer.split("\n")
    cleaned = []
    for line in lines:
        m = re.match(r"^\s*[-0-9.]*\s*([^\:]+)", line)
        if m:
            keyword = m.group(1).strip()
            cleaned.append(keyword)
    return "\n".join(cleaned)

# ---------------- 인덱스 구축: 법령(있으면) + ISMS-P 안내서 ----------------
all_docs: List[LawDoc] = []

# (선택) 기존 LAW_CONFIG 처리
for law_id, cfg in LAW_CONFIG.items():
    if not os.path.exists(cfg["pdf_path"]):
        print(f"[WARN] PDF not found: {cfg['pdf_path']}")
        continue
    docs = preprocess_law(law_id, cfg)
    all_docs.extend(docs)
    print(f"[OK] {cfg['law_name']} → chunks: {len(docs)}")

# ISMS-P 안내서 처리
if os.path.exists(GUIDE_PDF_PATH):
    isms_docs = preprocess_isms_p_guide(GUIDE_PDF_PATH)
    all_docs.extend(isms_docs)
    print(f"[OK] ISMS-P 안내서 → chunks: {len(isms_docs)}")
else:
    print(f"[WARN] ISMS-P 안내서 미발견: {GUIDE_PDF_PATH}")

indices = build_indices(all_docs)
print("[OK] built indices:", list(indices.keys()))
# 예상 키: 'faiss_hnsw_isms_p_guide', 'faiss_hnsw_all', (...법령 인덱스들)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/tmp/ipykernel_1868/2559704931.py:72: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be r

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

[WARN] ISMS-P 안내서 미발견: /mnt/data/금융보안원 - 금융권에 적합한 ISMS-P 인증기준 점검항목 안내서(2023.12.).pdf


IndexError: tuple index out of range